In [ ]:
import os
from os import path

# Third-party
from astropy.io import fits
from astropy.stats import median_absolute_deviation
from astropy.table import Table, join
from astropy.time import Time
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import h5py
import pandas as pd
from sqlalchemy import func
import tqdm

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import unimodal_P, MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold, _RV_LBL
from twoface.mass import get_m2_min, mf

In [ ]:
plot_path = '../../paper/1-catalog/figures/'
table_path = '../../paper/1-catalog/tables/'

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')
mcmc_samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter-mcmc.hdf5')

In [ ]:
run = session.query(JokerRun).limit(1).one()
joker_pars = run.get_joker_params()

In [ ]:
high_K_stars = session.query(AllStar).join(StarResult).filter(StarResult.status_id>0).filter(StarResult.high_K).all()
len(high_K_stars)

For all high-K stars, classify as unimodal or not based on TheJoker samples. Then do same for MCMC samples, AND the selections:

In [ ]:
unimodal_thejoker = []
with h5py.File(samples_file, 'r') as f:
    for star in tqdm.tqdm(high_K_stars):
        samples = JokerSamples.from_hdf5(f[star.apogee_id])

        data = star.apogeervdata()
        unimodal_thejoker.append(unimodal_P(samples, data))

unimodal_thejoker = np.array(unimodal_thejoker)
unimodal_thejoker.sum()

In [ ]:
unimodal_mcmc = []
converged_mcmc = []
with h5py.File(mcmc_samples_file, 'r') as f:
    for star in tqdm.tqdm(high_K_stars):
        if star.apogee_id not in f: 
            unimodal_mcmc.append(False)
            converged_mcmc.append(True)
            continue
        
        R = f[star.apogee_id]['chain-stats/gelman_rubin'][:]
        converged_mcmc.append(np.mean(R) <= 1.1)
        
        samples = JokerSamples.from_hdf5(f[star.apogee_id])

        data = star.apogeervdata()
        unimodal_mcmc.append(unimodal_P(samples, data))
        
unimodal_mcmc = np.array(unimodal_mcmc)
converged_mcmc = np.array(converged_mcmc)
unimodal_mcmc.sum(), converged_mcmc.sum()

In [ ]:
unimodal_mask = unimodal_thejoker | unimodal_mcmc
unimodal_converged_mask = unimodal_thejoker & (unimodal_mcmc & converged_mcmc)
unimodal_converged_idx = np.where(unimodal_converged_mask)[0]
unimodal_mask.sum(), unimodal_converged_mask.sum()

In [ ]:
np.random.seed(123)

rc = {
    'axes.labelsize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14
}
    
rand_subset = np.random.choice(unimodal_converged_idx, size=8, 
                               replace=False)

with mpl.rc_context(rc):
    fig, axes = plt.subplots(4, 2, figsize=(8, 10), sharex=True)

    for i, idx in enumerate(rand_subset):
        ax = axes.flat[i]

        star = high_K_stars[idx]
        data = star.apogeervdata()

        if unimodal_mcmc[idx] & converged_mcmc[idx]:
            fn = mcmc_samples_file
            print('mcmc')
        else:
            fn = samples_file
            print('thejoker')

        with h5py.File(fn, 'r') as f:
            samples = JokerSamples.from_hdf5(f[star.apogee_id])

        samples.t0 = data.t0
    
        sample = MAP_sample(data, samples, joker_pars)
        fig = plot_phase_fold(data, sample, ax=ax, 
                              jitter_errorbar=True, label=False)
        xlim = ax.get_xlim()
        ylim = (data.rv.value.min(), data.rv.value.max())
        yspan = ylim[1]-ylim[0]
        ylim = ax.set_ylim(ylim[0]-0.35*yspan, ylim[1]+0.35*yspan)

        text = ('{0},  '.format(star.apogee_id) + 
                '$P = {0.value:.2f}$ {0.unit:latex},  '.format(sample['P']) + 
                '$e = {0:.2f}$'.format(sample['e']))
        ax.text(xlim[0] + (xlim[1]-xlim[0])/15,
                ylim[1] - (ylim[1]-ylim[0])/20,
                text, fontsize=10, va='top', ha='left')
        # _ = plot_two_panel(data, samples)

    ax.set_xlim(-0.02, 1.02)

    for i in [0,1]:
        axes[-1, i].set_xlabel(r'phase, $\frac{M-M_0}{2\pi}$')

    for i in range(4):
        axes[i, 0].set_ylabel(_RV_LBL.format(u.km/u.s))

    fig.suptitle('Example stars from the high-$K$, unimodal sample', 
                 x=0.55, y=0.96, fontsize=18)
    fig.tight_layout()
    fig.subplots_adjust(top=0.92)
    fig.savefig(path.join(plot_path, 'highK-unimodal.pdf'))

## Produce catalog table:

In [ ]:
unimodal_stars = np.array(high_K_stars)[unimodal_mask]
unimodal_converged = converged_mcmc[unimodal_mask]

In [ ]:
rows = dict()
rows['APOGEE_ID'] = []
for k in JokerSamples._valid_keys:
    rows[k] = []
    rows[k + '_err'] = []
rows['t0'] = []
rows['emcee_converged'] = []
rows['Gelman-Rubin'] = []

with h5py.File(mcmc_samples_file, 'r') as mcmc_f, h5py.File(samples_file, 'r') as joker_f:
    for i, star in tqdm.tqdm(enumerate(unimodal_stars)):
        data = star.apogeervdata()
        if star.apogee_id in mcmc_f:
            samples = JokerSamples.from_hdf5(mcmc_f[star.apogee_id])
            R = mcmc_f[star.apogee_id]['chain-stats/gelman_rubin'][:]
        else:
            samples = JokerSamples.from_hdf5(joker_f[star.apogee_id])
            R = np.full(7, np.nan)
        
        rows['APOGEE_ID'].append(star.apogee_id)
        MAP = MAP_sample(data, samples, joker_pars)
        for k in samples.keys():
            rows[k].append(MAP[k])
            rows[k+'_err'].append(1.5 * median_absolute_deviation(samples[k]))
            
        rows['t0'].append(data.t0.tcb.mjd)
        rows['emcee_converged'].append(unimodal_converged[i])
        rows['Gelman-Rubin'].append(R)
        
for k in rows:
    if hasattr(rows[k][0], 'unit'):
        rows[k] = u.Quantity(rows[k])
        
rows['t0'] = Time(rows['t0'], format='mjd', scale='tcb')

In [ ]:
tbl = Table(rows)

### Add Ness masses to table:

In [ ]:
ness_tbl = Table.read('../../data/NessRG.fits')
ness_tbl.rename_column('2MASS', 'APOGEE_ID')
ness_tbl = ness_tbl[np.isin(ness_tbl['APOGEE_ID'], tbl['APOGEE_ID'])]

# trim the duplicates...
_, unq_idx = np.unique(ness_tbl['APOGEE_ID'], return_index=True)
ness_tbl = ness_tbl[unq_idx]

tbl_with_ness = join(tbl, ness_tbl, keys='APOGEE_ID', join_type='outer')
assert len(tbl_with_ness) == len(tbl)

### Compute m2_min using Ness mass

In [ ]:
rnd = np.random.RandomState(seed=42)
N = rnd.normal

mass_ratio = np.full(len(tbl_with_ness), np.nan)
mass_ratio_err = np.full(len(tbl_with_ness), np.nan)
m1 = np.full(len(tbl_with_ness), np.nan) * u.Msun
m1_err = np.full(len(tbl_with_ness), np.nan) * u.Msun
m2_min = np.full(len(tbl_with_ness), np.nan) * u.Msun
m2_min_err = np.full(len(tbl_with_ness), np.nan) * u.Msun

n_samples = 1024
for i, row in tqdm.tqdm(enumerate(tbl_with_ness)):
    if tbl_with_ness['lnM'].mask[i]:
        continue
        
    m1_samples = np.exp(N(row['lnM'], row['e_logM'], size=n_samples)) * u.Msun
    mass_func = mf(P=N(row['P'], row['P_err'], n_samples) * tbl_with_ness['P'].unit, 
                   K=N(row['K'], row['K_err'], n_samples) * tbl_with_ness['K'].unit,
                   e=N(row['e'], row['e_err'], n_samples))
    m2_mins = get_m2_min(m1_samples, mass_func)

    med_m1 = np.median(m1_samples)
    med_m2 = np.median(m2_mins)
    
    m1[i] = med_m1
    m1_err[i] = 1.5*median_absolute_deviation(m1_samples)
    m2_min[i] = med_m2
    m2_min_err[i] = 1.5*median_absolute_deviation(m2_mins)
    mass_ratio[i] = (med_m2 / med_m1).decompose().value
    mass_ratio_err[i] = 1.5*median_absolute_deviation(m2_mins / m1_samples).decompose().value
    
tbl_with_ness['M1'] = m1
tbl_with_ness['M1_err'] = m1_err
tbl_with_ness['M2_min'] = m2_min
tbl_with_ness['M2_min_err'] = m2_min_err

tbl_with_ness['M1'].mask = np.isnan(tbl_with_ness['M1'])
tbl_with_ness['M1_err'].mask = np.isnan(tbl_with_ness['M1'])
tbl_with_ness['M2_min'].mask = np.isnan(tbl_with_ness['M1'])
tbl_with_ness['M2_min_err'].mask = np.isnan(tbl_with_ness['M1'])

### Now we load the APOGEE AllStar table to join the APOGEE data with our orbits:

In [ ]:
allstar_tbl = fits.getdata('/Users/adrian/data/APOGEE_DR14/allStar-l31c.2.fits')
allstar_tbl = allstar_tbl[np.isin(allstar_tbl['APOGEE_ID'], tbl['APOGEE_ID'])]

# trim the duplicates...
_, unq_idx = np.unique(allstar_tbl['APOGEE_ID'], return_index=True)
allstar_tbl = allstar_tbl[unq_idx]
assert len(allstar_tbl) == len(tbl)

allstar_tbl = Table(allstar_tbl)
allstar_tbl.rename_column('K', 'KS')
allstar_tbl.rename_column('K_ERR', 'KS_ERR')

In [ ]:
full_catalog = join(tbl_with_ness, allstar_tbl, keys='APOGEE_ID')
full_catalog[:1]

### Finally, add a binary flag "DR14RC" if in DR14 RC catalog

In [ ]:
rc = Table.read('/Users/adrian/data/APOGEE_DR14/apogee-rc-DR14.fits')

In [ ]:
full_catalog['DR14RC'] = np.isin(full_catalog['APOGEE_ID'], rc['APOGEE_ID'])

TODO: describe in README with data to use `Table.read('', astropy_native=True)`

In [ ]:
len(full_catalog)

In [ ]:
full_catalog['M1'][full_catalog['M1'].mask] = np.nan

In [ ]:
full_catalog.write(path.join(table_path, 'high-K-unimodal.fits'), overwrite=True)

In [ ]:
for name in full_catalog.colnames[:18] + full_catalog.colnames[33:37]:
    c1 = '\\texttt{{{0}}}'.format(name.replace('_', '\\_'))
    try:
        c2 = '{0:latex_inline}'.format(full_catalog[name].unit)
    except TypeError:
        c2 = ''
    except AttributeError:
        c2 = ''
    
    if len(c1) < 26:
        c1 = c1 + ' '*(26 - len(c1))
        
    if len(c2) < 24:
        c2 = c2 + ' '*(24 - len(c2))
        
    print('{0} & {1} & <description> \\\\'.format(c1, c2))

---

In [ ]:
emcee_converged = full_catalog[full_catalog['emcee_converged']]

In [ ]:
_path = '../../plots/emcee_converged'
os.makedirs(_path, exist_ok=True)

In [ ]:
with h5py.File(mcmc_samples_file, 'r') as mcmc_f, h5py.File(samples_file, 'r') as f:
    for row in emcee_converged:
        star = AllStar.get_apogee_id(session, row['APOGEE_ID'])
        data = star.apogeervdata()
    
        if star.apogee_id in mcmc_f:
            samples = JokerSamples.from_hdf5(mcmc_f[star.apogee_id])
            print('mcmc')
        else:
            samples = JokerSamples.from_hdf5(f[star.apogee_id])
            print('thejoker')
            
        samples.t0 = data.t0
        
        fig = plot_two_panel(data, samples, 
                             plot_data_orbits_kw=dict(n_times=16384,                
                                                      highlight_P_extrema=False))
        fig.axes[0].set_title(star.apogee_id)
        fig.tight_layout()
        fig.savefig(path.join(_path, '{0}.png'.format(star.apogee_id)), dpi=200)
        plt.close(fig)

By-eye vetting: these ones are suspicious

In [ ]:
suspicious_ids = ['2M05224382+4300425',
                  '2M08505498+1156503',
                  '2M10264342+1340172',
                  '2M10513288-0250550',
                  '2M14574438+2106271',
                  '2M16131259+5043080',
                  '2M17121495+3211467',
                  '2M17212080+6003296',
                  '2M18571262-0328064',
                  '2M21260907+1100178',
                  '2M21374395+4304268']

In [ ]:
derp = emcee_converged[~np.isin(emcee_converged['APOGEE_ID'], suspicious_ids)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.errorbar(derp['P'], derp['LOGG'],
            xerr=derp['P_err'], yerr=derp['LOGG_ERR'],
            marker='o', linestyle='none', alpha=0.8)

ax.set_xscale('log')
ax.set_ylim(4., 0)
ax.set_xlabel('P')
ax.set_ylabel('logg')

# -----

fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.errorbar(derp['P'], derp['e'],
            xerr=derp['P_err'], yerr=derp['e_err'],
            marker='o', linestyle='none', alpha=0.8)

ax.set_xscale('log')
ax.set_ylim(0, 1)
ax.set_xlabel('P')
ax.set_ylabel('e')

# -----

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

ax = axes[0]
ax.errorbar(derp['M1'], derp['M2_min']/derp['M1'],
            xerr=derp['M1_err'], yerr=np.sqrt(derp['M1_err']**2+derp['M2_min_err']**2),
            marker='o', linestyle='none', alpha=0.8)
ax.set_xlabel('M1')
ax.set_ylabel('M2/M1')

ax = axes[1]
mass_ratio = derp['M2_min']/derp['M1']
ax.hist(mass_ratio[np.isfinite(mass_ratio)], bins='auto')
ax.set_xlabel('M2/M1')

In [ ]:
with h5py.File(mcmc_samples_file, 'r') as mcmc_f, h5py.File(samples_file, 'r') as f:
    for row in derp[rc_mask & (derp['P'] < 20)]:
        star = AllStar.get_apogee_id(session, row['APOGEE_ID'])
        data = star.apogeervdata()
    
        if star.apogee_id in mcmc_f:
            samples = JokerSamples.from_hdf5(mcmc_f[star.apogee_id])
            print('mcmc')
        else:
            samples = JokerSamples.from_hdf5(f[star.apogee_id])
            print('thejoker')
            
        samples.t0 = data.t0
        
        fig = plot_two_panel(data, samples, 
                             plot_data_orbits_kw=dict(n_times=16384,                
                                                      highlight_P_extrema=False))
        fig.axes[0].set_title('P = {0:.2f}'.format(samples['P'][0]))
        fig.tight_layout()

In [ ]:
derp[rc_mask & (derp['P'] < 20)]